In [12]:
import mlflow
import yaml
import os
import joblib
import json
import pandas as pd
import requests

from pgsqlconn import fetch_data
from my_helpers import print_types, transform_data

def load_data(path, loader):
    with open(path, 'r') as file:
        return  loader(file)

def check_mlflow_server(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            print("MLFlow server is running.")
            return True
        else:
            print("MLFlow server returned a status:", response.status_code)
            return False
    except requests.exceptions.ConnectionError:
        print("Failed to connect to the MLFlow server.")
        return False

project_path = './mlflow_server'
path_to_params = project_path + '/models/params.yaml'
path_to_model = project_path + '/models/fitted_model.pkl'
path_to_metrics = project_path + '/results/cv_res.json'

params = load_data(path_to_params, yaml.safe_load)
model = joblib.load(path_to_model)
metrics = load_data(path_to_metrics, json.load)
X_test = pd.read_csv(project_path + '/data/x_test.csv')

TABLE_NAME = "clean_flats"
REGISTRY_MODEL_NAME = 'model_sprint_2'
EXPERIMENT_NAME = 'Спринт 3/9: 2 спринт → Тема 5/5: Проект'
RUN_NAME = "ETL"

if check_mlflow_server(os.getenv('TRACKING_SERVER_CONN')):
    mlflow.set_tracking_uri(os.getenv('TRACKING_SERVER_CONN'))

    prediction = model.predict(X_test)
    pip_requirements = 'requirements.txt'
    signature = mlflow.models.infer_signature(X_test, prediction)
    input_example = X_test[:10]

    experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
    if not experiment:
        experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)
    else:
        experiment_id = experiment.experiment_id

    with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
        run_id = run.info.run_id
        mlflow.sklearn.log_model(sk_model=model, 
            artifact_path='models', 
            registered_model_name=REGISTRY_MODEL_NAME, 
            signature=signature, 
            input_example = input_example, 
            await_registration_for=60, 
            pip_requirements=pip_requirements
            )
        mlflow.log_metrics(metrics)
        mlflow.log_params(params)

    experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
    run = mlflow.get_run(run_id)

    print(f'Experiment "{experiment.name}" stage: {experiment.lifecycle_stage}')
    print(f'Run "{run.info.run_name}", id={run.info.run_id} status: {run.info.status}')

MLFlow server is running.


/home/mle-user/mle-project/mle-mlflow/mle-project-sprint-2-v001/.venv_sprint_2/lib/python3.10/site-packages/mlflow/types/utils.py:394: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
Registered model 'model_sprint_2' already exists. Creating a new version of this model...
2024/04/26 09:36:45 INFO mlflow.store.model_registry.abstract_store: Waiting up to 60 seco

CPU times: user 1.14 s, sys: 24 ms, total: 1.16 s
Wall time: 1.52 s


Created version '4' of model 'model_sprint_2'.


Experiment "Спринт 3/9: 2 спринт → Тема 5/5: Проект" stage: active
Run "ETL", id=97e36a4c008244f594b8213adc656d15 status: FINISHED
